In [259]:
import re
import glob
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords

from collections import Counter

import requests
from bs4 import BeautifulSoup as BS
from tqdm.notebook import tqdm
import codecs
import html5lib
from IPython.core.display import HTML

from spacy.lang.en import English
import spacy

from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
from spacy import displacy

import random
from spacy.util import minibatch, compounding

In [7]:
globs = glob.glob('../data/*.htm') + glob.glob('../data/*.html')

In [143]:
print(globs[11])
# test = open(globs[11], encoding="uft8")
# BS(test)

../data\mtn-20210731.htm


In [141]:
globs[11]
with open('../data\mtn-20210731.htm', encoding='utf8') as fi:
    file_html = fi.read()
file_html

'<?xml version="1.0"?>\n<html lang="en-US">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<title>mtn-20210731</title>\n\n<style type="text/css">\n table.mtn20210731htm583 {border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:94.005%}\n div.mtn20210731htm582 {margin-bottom:5pt;margin-top:5pt;text-align:center}\n td.mtn20210731htm581 {width:52.866%}\n td.mtn20210731htm580 {width:44.934%}\n table.mtn20210731htm579 {border-collapse:collapse;display:inline-table;margin-bottom:5pt;vertical-align:text-bottom;width:87.134%}\n td.mtn20210731htm578 {width:51.250%}\n td.mtn20210731htm577 {width:3.597%}\n td.mtn20210731htm576 {width:41.853%}\n td.mtn20210731htm575 {background-color:#cceeff;padding:2px 1pt;text-align:justify;vertical-align:top}\n td.mtn20210731htm574 {background-color:#ffffff;padding:2px 1pt;text-align:justify;vertical-align:top}\n div.mtn20210731htm573 {margin-bottom:5pt;margin-top:5pt;text-align:justify}\

In [169]:
companies = [re.findall(r'\\([a-z]+)', x)[0] for x in globs]
companies

['amrk',
 'cnxc',
 'earn',
 'ful',
 'gwre',
 'info',
 'intc',
 'kbh',
 'lvis',
 'mcft',
 'mdc',
 'mtn',
 'nflx',
 'noc',
 'panw',
 'prgs',
 'pstg',
 'siri',
 'slb',
 'tmb',
 'uri',
 'apog',
 'azz',
 'bbw',
 'ctrn',
 'fnhc',
 'kmx',
 'lake',
 'tlys',
 'vrrm']

In [148]:
nlp = spacy.load('en_core_web_sm')

In [161]:
# def remove_tags(htmlFile):
  
#     # parse html content
#     soup = BS(htmlFile)
  
#     for data in soup(['style', 'script']):
#         # Remove tags
#         data.decompose()
  
#     # return data by retrieving the tag content
#     return ' '.join(soup.stripped_strings)

htmlFile = open(globs[0])

soup = BS(htmlFile)

doc = nlp(soup.body.text)

sentences = [sent.orth_ for sent in doc.sents]

sentences

# sentences = sent_tokenize(soup.body.text)
sentences = [word for line in sentences for word in line.split("\n")]
repurchase_sent = [x for x in sentences if 'repurchas' in x]
repurchase_sent = [x for x in repurchase_sent if 'board' in x.lower()]
repurchase_sent = [x for x in repurchase_sent if re.search(r'\d*', x)]
repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]
repurchase_sent

['On April 26, 2018, the Company’s Board of Directors authorized a stock repurchase program for up to 500,000 shares of the Company’s stock.',
 '  The actual number of shares repurchased and the timing of repurchases will be determined by the Board of Directors and will depend on a number of factors, including stock price, trading volume, general market conditions, working capital requirements, general business conditions, and other factors.',
 "In April 2018, the Company's Board of Directors approved a share repurchase program which authorizes the Company to purchase up to 500,000 shares of its common stock from time to time, either in the open market or in block purchase transactions."]

In [189]:
i = globs[0]
with open(i, encoding='utf8') as fi:
        htmlFile = fi.read()
        soup = BS(htmlFile)

        doc = nlp(soup.body.text)

        sentences = [sent.orth_ for sent in doc.sents]

        sentences = [word for line in sentences for word in line.split("\n")]
        repurchase_sent = [x for x in sentences if 'repurchas' in x]
#         repurchase_sent = [x for x in repurchase_sent if 'board' in x.lower()]
        repurchase_sent = [x for x in repurchase_sent if re.search(r'\d+', x)]
        repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]

        sentence_embeddings = model.encode(repurchase_sent)

repurchase_sent
#         results = cosine_similarity([model_embeddings[0]],
#                                    sentence_embeddings[0:])
# print(i)
# print(results)

['The table below sets forth the cash dividends declared and paid in 2021, 2020 and 2019Date ofDeclarationDate ofPaymentDividendper ShareTotalDividendsPaid(In thousands)2021First Quarter01/25/2102/24/21$0.37 $25,978 Second Quarter04/26/2105/26/210.40 28,249 Third Quarter07/26/2108/25/210.40 28,276 Fourth Quarter10/25/2111/24/210.50 35,339 $1.67 $117,842 2020First Quarter01/27/2002/26/20$0.31 $20,768 Second Quarter04/01/2005/20/200.31 20,914 Third Quarter07/27/2008/26/200.31 21,374 Fourth Quarter10/26/2011/24/200.37 25,952 $1.29 $89,008 2019First Quarter01/28/1902/27/19$0.26 $17,019 Second Quarter04/29/1905/29/190.28 18,521 Third Quarter07/22/1908/21/190.28 18,700 Fourth Quarter10/28/1911/27/190.28 18,780 $1.09 $73,020 14Table of ContentsThe following table provides information about our repurchases of common stock during the Three Months Ended December 31, 2021:',
 'Period:Total Number of Shares Purchased (1)AveragePrice PaidPer ShareTotal Number of Shares Purchasedas Part of Publicly 

### BERT trial

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [3]:
model_sentence = ["In December 2018, the Company’s Board of Directors authorized an additional share repurchase program under which the Company may repurchase up to $10.0 million of its outstanding shares of common stock through December 31, 2019."]

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [164]:
model_embeddings = model.encode(model_sentence)
sentence_embeddings_one.shape

(1, 768)

In [205]:
indexes = []
best_sentences = []
cos_sim_score = []

for i in globs:
    print(i)
    with open(i, encoding='utf8') as fi:
        htmlFile = fi.read()
    soup = BS(htmlFile)
    
    doc = nlp(soup.body.text)

    sentences = [sent.orth_ for sent in doc.sents]
    
    sentences = [word for line in sentences for word in line.split("\n")]
    repurchase_sent = [x for x in sentences if 'repurchas' in x]
#     repurchase_sent = [x for x in repurchase_sent if 'program' in x.lower()]
    repurchase_sent = [x for x in repurchase_sent if re.search(r'\d*', x)]
    repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]
    
    sentence_embeddings = model.encode(repurchase_sent)
    
    results = cosine_similarity([model_embeddings[0]],
                               sentence_embeddings[0:])
  
    
    results_list = results[0].tolist()
    
    high_score_index = results_list.index(max(results_list))
    
    indexes = indexes + [high_score_index]
    
    best_sentences = best_sentences + [repurchase_sent[high_score_index]]
    
    cos_sim_score = cos_sim_score + [results_list[high_score_index]]

../data\amrk-10k_20210630.htm


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


../data\cnxc-20211130.htm


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


../data\earn-20211231.htm


...

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


../data\lake_10k.html


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


../data\tlys-20220129.html
../data\vrrm-20211231.html


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [206]:
approved_repurchase = pd.DataFrame({
    'original_index': indexes, 'company': companies, 'sim_score': cos_sim_score, 'sentence':best_sentences
})

In [231]:
approved_repurchase

,original_index,company,sim_score,sentence
0,52,amrk,0.815112,"In April 2018, the Company's Board of Director..."
1,13,cnxc,0.840245,NOTE 17—STOCKHOLDERS’ EQUITY:Share repurchase ...
2,107,earn,0.830385,"In addition as of December 31, 2021 and 2020, ..."
3,18,ful,0.827594,"We repurchased 50,799, 70,380 and 73,043 share..."
4,3,gwre,0.869305,"In October 2020, our board of directors author..."
5,21,info,0.821637,Our Board of Directors authorized a share repu...
6,4,intc,0.856205,"In the first quarter of 2021, we repurchased t..."
7,24,kbh,0.853297,In addition to the shares purchased in 2021 p...
8,7,lvis,0.844833,"In October 2021, the Board reinstated its shar..."
9,13,mcft,0.829230,"On June 24, 2021, the board of directors autho..."


In [214]:
approved_repurchase['sentence'][0]

"In April 2018, the Company's Board of Directors approved a share repurchase program which authorizes the Company to purchase up to 500,000 shares of its common stock from time to time, either in the open market or in block purchase transactions."

#### Rohit code

In [266]:
# Read annotations into dataframe
annotations = pd.read_json('../data/annotations.json')

In [267]:
# Split annotations into individual sentences
s = (annotations.pop('text')
      .str.strip('.')
      .str.split('\.\s+', expand = True)
      .stack()
      .rename('text')
      .reset_index(level = 1, drop = True)) + '.'
annotations = annotations.join(s).reset_index(drop = True)

In [268]:
# Filter out this stubborn sentence
annotations = annotations.loc[~(annotations['text'] == 'See Part II, Item 5 of this Report and Note 6 to the Financial Statements for more information.')]

In [270]:
# Write entity-specific sentences to csv
annotations.loc[annotations.apply(lambda x: str(x['authorization_date']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('authorization_date').to_csv('../data/authorization_date.csv', index = False)
annotations.loc[annotations.apply(lambda x: str(x['authorization_amount']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('authorization_amount').to_csv('../data/authorization_amount.csv', index = False)
annotations.loc[annotations.apply(lambda x: str(x['repurchase_date']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('repurchase_date').to_csv('../data/repurchase_date.csv', index = False)
annotations.loc[annotations.apply(lambda x: str(x['repurchase_number']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('repurchase_number').to_csv('../data/repurchase_number.csv', index = False)
annotations.loc[annotations.apply(lambda x: str(x['repurchase_amount']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('repurchase_amount').to_csv('../data/repurchase_amount.csv', index = False)
annotations.loc[annotations.apply(lambda x: str(x['authorization_number']) in x['text'], axis = 1), 'text'].drop_duplicates().rename('authorization_number').to_csv('../data/authorization_number.csv', index = False)

In [271]:
# Read in entity-specific sentences as lists
authorization_date = pd.read_csv('../data/authorization_date.csv')['authorization_date'].tolist()
authorization_amount = pd.read_csv('../data/authorization_amount.csv')['authorization_amount'].tolist()
repurchase_date = pd.read_csv('../data/repurchase_date.csv')['repurchase_date'].tolist()
repurchase_number = pd.read_csv('../data/repurchase_number.csv')['repurchase_number'].tolist()
repurchase_amount = pd.read_csv('../data/repurchase_amount.csv')['repurchase_amount'].tolist()
authorization_number = pd.read_csv('../data/authorization_number.csv')['authorization_number'].tolist()

In [277]:
auth_date_embeddings = model.encode(authorization_date)
auth_amount_embeddings = model.encode(authorization_amount)
rep_date_embeddings = model.encode(repurchase_date)
rep_numb_embeddings = model.encode(repurchase_number)
rep_amount_embeddings = model.encode(repurchase_amount)
auth_numb_embeddings = model.encode(authorization_number)

In [304]:
len(authorization_number)

13

In [348]:
best_sentences = []
max_scores = []
for i in globs:
    with open(i, encoding='utf8') as fi:
            htmlFile = fi.read()
            soup = BS(htmlFile)

            doc = nlp(soup.body.text)

            sentences = [sent.orth_ for sent in doc.sents]

            sentences = [word for line in sentences for word in line.split("\n")]
            repurchase_sent = [x for x in sentences if 'repurchas' in x]
            repurchase_sent = [x for x in repurchase_sent if re.search(r'\d+', x)]
            repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]

            sentence_embeddings = model.encode(repurchase_sent)

            best_score = 0
            for foo in range(len(auth_date_embeddings)):
                results = cosine_similarity([auth_date_embeddings[foo]],
                                       sentence_embeddings[0:])
                results_list = results[0].tolist()

                high_score_index = results_list.index(max(results_list))
                max_score = max(results_list)
                if max_score > best_score:
                    best_score = max_score
                    sentence = repurchase_sent[high_score_index]
                else:
                    pass
    best_sentences = best_sentences + [sentence]
    max_scores = max_scores + [best_score]


C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

In [354]:
auth_date_df = pd.DataFrame({
    'company': companies,
    'auth_date_score': max_scores,
    'auth_date_sentence': best_sentences
})

auth_date_df.to_csv('../data/authorization_date_sentences.csv')

In [360]:
def similarity_pipeline(filepaths, embeddings):
    best_sentences = []
    max_scores = []
    for i in filepaths:
        with open(i, encoding='utf8') as fi:
                htmlFile = fi.read()
                soup = BS(htmlFile)

                doc = nlp(soup.body.text)

                sentences = [sent.orth_ for sent in doc.sents]

                sentences = [word for line in sentences for word in line.split("\n")]
                repurchase_sent = [x for x in sentences if 'repurchas' in x]
                repurchase_sent = [x for x in repurchase_sent if re.search(r'\d+', x)]
                repurchase_sent = [x.replace('\xa0', " ") for x in repurchase_sent]

                sentence_embeddings = model.encode(repurchase_sent)

                best_score = 0
                for foo in range(len(embeddings)):
                    results = cosine_similarity([embeddings[foo]],
                                           sentence_embeddings[0:])
                    results_list = results[0].tolist()

                    high_score_index = results_list.index(max(results_list))
                    max_score = max(results_list)
                    if max_score > best_score:
                        best_score = max_score
                        sentence = repurchase_sent[high_score_index]
                    else:
                        pass
        best_sentences = best_sentences + [sentence]
        max_scores = max_scores + [best_score]
        companies = [re.findall(r'\\([a-z]+)', x)[0] for x in filepaths]
        
    df = pd.DataFrame({
        'company': companies,
        'cos_sim_score': max_scores,
        'sentence': best_sentences
    })
        
    return df


In [361]:
auth_amount_df = similarity_pipeline(globs, auth_amount_embeddings)
auth_amount_df.to_csv('../data/auth_amount_sentences.csv')

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

,company,cos_sim_score,sentence
0,amrk,0.947255,"In April 2018, the Company's Board of Director..."
1,cnxc,0.980644,NOTE 17—STOCKHOLDERS’ EQUITY:Share repurchase ...
2,earn,0.911583,"On June 13, 2018, our Board of Trustees approv..."
3,ful,1.000000,"On April 6, 2017, the Board of Directors autho..."
4,gwre,1.000000,"In October 2020, our board of directors author..."
5,info,1.000000,"In addition, the Board previously authorized a..."
6,intc,1.000000,"As of December 25, 2021, we were authorized to..."
7,kbh,0.954627,"On July 8, 2021, our board of directors autho..."
8,lvis,1.000000,"In October 2021, the Board reinstated its shar..."
9,mcft,1.000000,"On June 24, 2021, the board of directors autho..."


In [363]:
rep_date_df = similarity_pipeline(globs, rep_date_embeddings)
rep_date_df.to_csv('../data/rep_date_sentences.csv')

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

In [364]:
rep_numb_df = similarity_pipeline(globs, rep_numb_embeddings)
rep_numb_df.to_csv('../data/rep_numb_sentences.csv')

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

In [365]:
rep_amount_df = similarity_pipeline(globs, rep_amount_embeddings)
rep_amount_df.to_csv('../data/rep_amount_sentences.csv')

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

In [366]:
auth_numb_df = similarity_pipeline(globs, auth_numb_embeddings)
auth_numb_df.to_csv('../data/auth_numb_sentences.csv')

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into 

In [ ]:
def prefix_addition(df, prefix):
    df.rename(columns = {'cos_sim_score': (prefix + ' cos_sim_score'), 'sentence':(prefix + ' sentence')})